In [1]:
import torch
import uuid
import os
import numpy as np
from datasets import Dataset
from transformers import (
    T5ForConditionalGeneration,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments
)

Из-за того, что мы создаем датасет с помощью матчинга негативных текстов в какой-либо текст из позитивной или нейтральной группы, неизбежно появляются ошибки связанные с тем, что в оригинальном тексте и таргете, может в принципе говориться про разные продукты.

Чтобы хоть как-то передать информацию о желаемым результатом, я предлагаю включить в промт информацию о продукте и типу текста

In [2]:
ds = Dataset.from_parquet('./__output__/mathced.parquet')
ds = ds.select(np.random.randint(0, len(ds), 2000)) # type: ignore

ds

Dataset({
    features: ['nmId', 'text', 'responder', 'type', 'product_name', 'product_category_2', 'product_category_1', 'product_color', 'product_description', 'product_brand', 'vector', 'toxicity', 'emotions', 'target_nmId', 'target_text', 'target_responder', 'target_type', 'target_product_name', 'target_product_category_2', 'target_product_category_1', 'target_product_color', 'target_product_description', 'target_product_brand', 'target_vector', 'target_toxicity', 'target_emotions'],
    num_rows: 2000
})

Так как у нас задача Seq2Seq модель, самым логичным вариантом будет выбрать какую-либо Encoder-Decoder модель, из которых самая популярная это flan-t5

In [3]:
model = T5ForConditionalGeneration.from_pretrained(
    'google/flan-t5-large',
    device_map='cuda:0',
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-small')

coll = DataCollatorForSeq2Seq(model=model, tokenizer=tokenizer)

In [4]:
def preprocess(e):
    out = tokenizer(
        f'''
responder: {e['target_responder']}
type: {e['target_type']}
product_name: {e['target_product_name']}
product_category_2: {e['target_product_category_2']}
product_color: {e['target_product_color']}
product_brand: {e['target_product_brand']}
product_description: {e['target_product_description']}
toxicity: {e['toxicity']}
emotions: {e['emotions']}
text: {e['text']}
        ''',
        truncation=True,
        max_length=tokenizer.model_max_length
    )

    labels = tokenizer(
        text_target=e['target_text'], 
        max_length=512,         
        truncation=True
    )
    out['labels'] = labels['input_ids']
    
    return out

In [5]:
ds = ds.map(preprocess, remove_columns=ds.column_names) # type: ignore
ds = ds.train_test_split(test_size=0.1) # type: ignore

ds

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1800
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [6]:
checkpoint = str(uuid.uuid4())

checkpoint

'fd81905c-e7e0-40e8-ae45-731658cd9780'

In [7]:
os.makedirs(f'./models/{checkpoint}')

In [8]:
args = TrainingArguments(
    output_dir=f'./models/{checkpoint}/runs',
    eval_strategy='epoch',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    learning_rate=0.02,
    warmup_ratio=0.1,
    weight_decay=0.01,
    save_strategy='no',
    # fp16=True
)

In [9]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer, # type: ignore
    args=args,
    data_collator=coll,
    train_dataset=ds['train'], # type: ignore
    eval_dataset=ds['test'] # type: ignore
)

C:\Users\roaoc\AppData\Local\Temp\ipykernel_6824\2017768784.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()

  0%|          | 0/27000 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [ ]:
trainer.save_model(f'./models/{checkpoint}/model')